## <font>Air Quality Data in India (2015 - 2020) for Time Series problem</font>
https://www.kaggle.com/rohanrao/air-quality-data-in-india

In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
city_day = pd.read_csv('city_day.csv')
city_day1=city_day.copy(deep=True)
city_day.head()

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
1,Ahmedabad,2015-01-02,NaN,NaN,0.97,15.69,16.46,NaN,0.97,24.55,34.06,3.68,5.50,3.77,NaN,NaN
2,Ahmedabad,2015-01-03,NaN,NaN,17.40,19.30,29.70,NaN,17.40,29.07,30.70,6.80,16.40,2.25,NaN,NaN
3,Ahmedabad,2015-01-04,NaN,NaN,1.70,18.48,17.97,NaN,1.70,18.59,36.08,4.43,10.14,1.00,NaN,NaN
4,Ahmedabad,2015-01-05,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.33,39.31,7.01,18.89,2.78,NaN,NaN


In [10]:
city_day.describe()

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI
count,29531.000000,29531.000000,29531.000000,29531.000000,29531.000000,29531.000000,29531.000000,29531.000000,29531.000000,29531.000000,29531.000000,29531.000000,29531.000000
mean,66.137399,97.578733,17.201890,26.909513,32.611069,22.650619,2.292734,15.650236,34.130603,3.130408,7.027591,2.864628,162.909011
std,64.101785,84.860683,23.245966,23.671982,33.829601,25.794354,6.824266,20.583528,21.631257,15.988058,17.841550,5.383474,138.486775
min,0.040000,0.010000,0.020000,0.010000,0.000000,0.010000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,13.000000
25%,29.190000,31.580000,5.900000,11.630000,13.180000,8.680000,0.500000,5.740000,18.860000,0.050000,0.280000,0.180000,76.000000
50%,46.900000,72.380000,9.280000,19.720000,21.980000,15.150000,0.910000,8.950000,31.480000,0.670000,1.720000,1.250000,114.000000
75%,78.620000,138.605000,18.560000,35.030000,38.660000,26.640000,1.490000,15.840000,44.215000,2.750000,6.710000,2.695000,206.000000
max,949.990000,1000.000000,390.680000,362.210000,467.630000,352.890000,175.810000,193.860000,257.730000,455.030000,454.850000,170.370000,2049.000000


In [11]:
city_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29531 entries, 0 to 29530
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        29531 non-null  object 
 1   Date        29531 non-null  object 
 2   PM2.5       29531 non-null  float64
 3   PM10        29531 non-null  float64
 4   NO          29531 non-null  float64
 5   NO2         29531 non-null  float64
 6   NOx         29531 non-null  float64
 7   NH3         29531 non-null  float64
 8   CO          29531 non-null  float64
 9   SO2         29531 non-null  float64
 10  O3          29531 non-null  float64
 11  Benzene     29531 non-null  float64
 12  Toluene     29531 non-null  float64
 13  Xylene      29531 non-null  float64
 14  AQI         29531 non-null  float64
 15  AQI_Bucket  29531 non-null  object 
dtypes: float64(13), object(3)
memory usage: 3.6+ MB


In [3]:
missing_values_city_day = city_day.isnull().sum()
percentage_city_day = 100 * city_day.isnull().sum() / len(city_day)
missing_values_table_city_day = pd.concat([missing_values_city_day, percentage_city_day], axis=1)
missing_values_table_city_day.columns = ['Num. of missing values','% of missing values']
missing_values_table_city_day

,Num. of missing values,% of missing values
City,0,0.000000
Date,0,0.000000
PM2.5,4598,15.570079
PM10,11140,37.723071
NO,3582,12.129626
NO2,3585,12.139785
NOx,4185,14.171549
NH3,10328,34.973418
CO,2059,6.972334
SO2,3854,13.050692


There are a lot of missing values and some of the columns like **Xylene** and **PM10** have more than 50% of the values missing. Let's now see how we can impute these missing values.

In [4]:
city_day['Xylene'][60:70]

60    0.22
61    2.25
62    1.55
63    4.13
64     NaN
65    0.39
66    0.63
67    1.35
68    1.74
69    2.78
Name: Xylene, dtype: float64

In [5]:
city_day.fillna(method='ffill',inplace=True)
city_day['Xylene'][60:70]

C:\Users\agami\AppData\Local\Temp\ipykernel_37620\2703101539.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  city_day.fillna(method='ffill',inplace=True)


60    0.22
61    2.25
62    1.55
63    4.13
64    4.13
65    0.39
66    0.63
67    1.35
68    1.74
69    2.78
Name: Xylene, dtype: float64

In [8]:
city_day['AQI'][20:30]

20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
26      NaN
27      NaN
28    209.0
29    328.0
Name: AQI, dtype: float64

In [9]:
city_day.fillna(method='bfill',inplace=True)
city_day['AQI'][20:30]

C:\Users\agami\AppData\Local\Temp\ipykernel_37620\4086047996.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  city_day.fillna(method='bfill',inplace=True)


20    209.0
21    209.0
22    209.0
23    209.0
24    209.0
25    209.0
26    209.0
27    209.0
28    209.0
29    328.0
Name: AQI, dtype: float64

### <font>Imputation using Linear Interpolation method</font>

In [6]:
city_day1['Xylene'][60:70]

60    0.22
61    2.25
62    1.55
63    4.13
64     NaN
65    0.39
66    0.63
67    1.35
68    1.74
69    2.78
Name: Xylene, dtype: float64

In [7]:
# Interpolate using the linear method
city_day1.interpolate(limit_direction="both",inplace=True)
city_day1['Xylene'][60:70]

C:\Users\agami\AppData\Local\Temp\ipykernel_37620\1666352788.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  city_day1.interpolate(limit_direction="both",inplace=True)


60    0.22
61    2.25
62    1.55
63    4.13
64    2.26
65    0.39
66    0.63
67    1.35
68    1.74
69    2.78
Name: Xylene, dtype: float64